# Gold Dataset Generator
<font size=3><strong>Author:</strong> <a href="https://www.linkedin.com/in/~ashkan/" target="_blank">Ashkan Soltanieh</a><br>
<strong>Date:</strong> Jan. 15, 2022</font>

## Table of Contents

<div class="alert alert-success mt-20">
    <ul>
        <li><a href="#Approach">Approach</a></li>
        <li><a href="#Merging Area of Burn and Characteristics Data">Merging Area of Burn and Characteristics Data</a></li>
        <li><a href="#Duplication Check in Merged Data">Duplication Check in Merged Data</a></li>
        <li><a href="#Dealing with Duplicates">Dealing with Duplicates</a></li>
        <li><a href="#Quartile Analysis">Quartile Analysis</a></li>
        <li><a href="#Metadata">Metadata</a></li>
    </ul>
</div>

## Approach:
So far we have merged and cleaned part of the data. As a quick overview, we have cleaned the two datasets for wildfires, as well as merged and cleaned weather data. Additionally, weather data are aggregated by their daily mean and standard deviation to bring consistency into dataset and alignment with wildfire datasets. The process of cleaning the weather data to create the focus weather dataset based on wildfire date and location and drop the rest of redundancy is completed, and silver dataset contains required aggregated variables for merging datasets.

In this notebook, our goal is to merge all dataset and start data preprocessing and exploratory analysis. The keys for merging Area of Burn and characteristics datasets are UID_Fire, and REF_ID, and Date_of_Burn. For merging wildfire and weather data we will be using rounded spatial locations data and date.

Finally, we are going to categorize the fire data into five categories. The categories will be selected based on quartile analysis for area of burn data. Ranking wildfire based on different citeria is a common practice. Here is an example of this has been done in Government of BC ([Wildfire Ranking](https://www2.gov.bc.ca/gov/content/safety/wildfire-status/about-bcws/wildfire-response/fire-characteristics/rank)).

In [1]:
import os
import numpy as np
import pandas as pd
import sys
from IPython.display import display
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(),"..","src/data")))
from weather import get_rounded_locations

In [2]:
path_weather = os.path.abspath(os.path.join(os.getcwd(), '../data/processed/weather/silver/silver_weather-daily-mean-std.csv'))
path_aob = os.path.abspath(os.path.join(os.getcwd(), '../data/processed/wildfire/silver/silver_AoB.csv'))
path_characteristics = os.path.abspath(os.path.join(os.getcwd(), '../data/processed/wildfire/silver/silver_chracteristics.csv'))

In [3]:
df_weather = pd.read_csv(path_weather)
df_aob = pd.read_csv(path_aob)
df_characteristics = pd.read_csv(path_characteristics)

## Merging Area of Burn and Characteristics Data

In [4]:
df_aob_merge = df_aob.rename(columns={'Date_of_Burn' : 'Date'}).set_index(['UID_Fire', 'REF_ID', 'Date'])
df_characteristics_merge = df_characteristics.set_index(['UID_Fire', 'REF_ID', 'Date'])
df_characteristics_aob = df_characteristics_merge.merge(df_aob_merge, on=['UID_Fire', 'REF_ID', 'Date'], how='inner')
df_characteristics_aob.reset_index(drop=False, inplace=True)
df_characteristics_aob.head(3)

,UID_Fire,REF_ID,Date,sat,Status,rounded_lat,rounded_lon,lat_mean,lat_std,lon_mean,lon_std,T21_mean,T21_std,T31_mean,T31_std,FRP_mean,FRP_std,conf_mean,conf_std,Total_AoB
0,334,BC-2010-G60081,2010-03-11,A,primary,55.50,-123.75,55.6140,0.000000,-123.8230,0.000000,38.55,0.000000,-6.75,0.000000,17.4,0.000000,0.0,0.000000,0.101448
1,334,BC-2010-G60081,2010-03-11,A,residual,55.50,-123.75,55.6140,0.000000,-123.8230,0.000000,38.55,0.000000,-6.75,0.000000,17.4,0.000000,0.0,0.000000,0.101448
2,676,AB-2010-SWF045,2010-03-19,A,primary,55.25,-116.75,55.2865,0.002121,-116.8635,0.012021,47.60,8.131728,-0.20,1.202082,26.5,9.192388,79.5,6.363961,0.019707


## Duplication Check in Merged Data
To check duplication after merging data we cannot use <code>rounded_lat</code> and <code>rounded_lon</code> data as the original area of burn data are not uniquely identified with any geospatioal data. On the other hand, Fire Status and Satlite data are needed in classification in order to distinguish the status type of the fire and which satelite is being used for detecting fire characteristics.

In [7]:
df_temp = df_characteristics_aob.set_index(['UID_Fire', 'REF_ID', 'Date', 'Total_AoB', 'Status', 'sat'])
display('Duplicated fire data after merging:', np.unique(df_temp.index.duplicated(), return_counts=True))

'Duplicated fire data after merging:'

(array([False,  True]), array([14184,  9614]))

## Dealing with Duplicates
Observing duplicated indices using <code>['UID_Fire', 'REF_ID', 'Date', 'Total_AoB', 'Status', 'sat']</code> shows that 9,614 number of the records have different rounded latitude and longitude (i.e. belonging to different quater geospatial degree grouping), while referring to the same fire incident. As a result to have more trust-worthy dataset we need to only keep one of the duplicated records and drop the rest. We will be keeping first observation of the duplicated records.

In [50]:
df_characteristics_aob = df_characteristics_aob.groupby(['UID_Fire', 'REF_ID', 'Date', 'Total_AoB', 'Status', 'sat'], 
                                                        axis=0, 
                                                        as_index=False).first()

As expected, no duplication observed after merging in the data.

In [51]:
display(df_aob.shape, df_aob.head(3))
display(df_characteristics.shape, df_characteristics.head(3))
display(df_characteristics_aob.shape, df_characteristics_aob.head(3))

(14891, 4)

,UID_Fire,REF_ID,Date_of_Burn,Total_AoB
0,100,BC-2011-V30040,2011-05-20,0.021933
1,100,BC-2014-G80090,2014-05-31,0.107307
2,1000,AB-2014-HWF124,2014-06-29,0.255955


(25442, 19)

,Date,sat,UID_Fire,Status,REF_ID,rounded_lat,rounded_lon,lat_mean,lat_std,lon_mean,lon_std,T21_mean,T21_std,T31_mean,T31_std,FRP_mean,FRP_std,conf_mean,conf_std
0,2010-01-12,A,313,removed,BC-2010-G40151,53.75,-124.25,53.829,0.000000,-124.3320,0.000000,69.75,0.000000,2.45,0.000000,110.7,0.000000,93.0,0.000000
1,2010-01-12,T,313,removed,BC-2010-G40151,53.75,-124.25,53.832,0.000000,-124.3350,0.000000,44.35,0.000000,0.15,0.000000,82.7,0.000000,64.0,0.000000
2,2010-01-13,A,313,removed,BC-2010-G40151,53.75,-124.25,53.838,0.001414,-124.3265,0.010607,71.90,8.980256,-3.55,0.424264,63.8,18.526198,81.5,3.535534


(14184, 20)

,UID_Fire,REF_ID,Date,Total_AoB,Status,sat,rounded_lat,rounded_lon,lat_mean,lat_std,lon_mean,lon_std,T21_mean,T21_std,T31_mean,T31_std,FRP_mean,FRP_std,conf_mean,conf_std
0,51,BC-2014-G40112,2014-10-24,0.245498,primary,T,54.25,-124.75,54.281,0.000000,-124.8510,0.000000,28.85,0.000000,2.85,0.000000,19.0,0.000000,50.0,0.000000
1,56,AB-2011-CWF240,2012-05-12,1.594830,primary,T,51.25,-115.00,51.160,0.005657,-114.8985,0.000707,38.65,0.707107,23.05,0.565685,13.3,0.989949,70.5,0.707107
2,68,BC-2014-C50011,2014-04-12,0.145104,primary,A,52.00,-123.50,52.099,0.000000,-123.4020,0.000000,29.85,0.000000,14.35,0.000000,8.3,0.000000,35.0,0.000000


Comparison between characteristics and area of burn data shows that 14,184 fire observations have a corresponding area of burn recording. These data will be further refined during exploratory analysis.

Here we're just dropping rounded_lat, and rounded_lon data as they're just creating for merging weather and fire tables

In [59]:
df_characteristics_aob_merge = df_characteristics_aob.set_index(['rounded_lat', 'rounded_lon', 'Date'])
df_weather_merge = df_weather.rename(columns = {'latitude': 'rounded_lat', 'longitude': 'rounded_lon', 'date':'Date'}).set_index(['rounded_lat', 'rounded_lon', 'Date'])
df_characteristics_aob_weather = df_characteristics_aob_merge.merge(df_weather_merge, on=['rounded_lat', 'rounded_lon', 'Date'], how='inner')
df_characteristics_aob_weather.reset_index(drop=False, inplace=True)
df_characteristics_aob_weather.set_index(['UID_Fire', 'REF_ID', 'Date'], inplace=True)
df_characteristics_aob_weather.drop(['rounded_lat', 'rounded_lon'], axis=1, inplace=True)

In [60]:
df_characteristics_aob_weather.columns

Index(['Total_AoB', 'Status', 'sat', 'lat_mean', 'lat_std', 'lon_mean',
       'lon_std', 'T21_mean', 'T21_std', 'T31_mean', 'T31_std', 'FRP_mean',
       'FRP_std', 'conf_mean', 'conf_std', 't2m_mean', 't2m_std', 'cape_mean',
       'cape_std', 'd2m_mean', 'd2m_std', 'tp_mean', 'tp_std', 'tcc_mean',
       'tcc_std', 'cvh_mean', 'cvl_mean', 'swvl1_mean', 'swvl1_std',
       'wind_speed_mean', 'wind_speed_std'],
      dtype='object')

## Quartile Analysis

We couldn't find any categorizing only based on wildfire burn area from the available references and litrature review. Therefore, since the current fire observations are high enough we decided to do fair distribution into five categories using quartile analysis based on quantile limits of Total_AoB data.

In [61]:
Q1 = df_characteristics_aob_weather['Total_AoB'].quantile(0.25)
median = df_characteristics_aob_weather['Total_AoB'].quantile(0.5)
Q3 = df_characteristics_aob_weather['Total_AoB'].quantile(0.75)
lower_extreme = Q1 - 1.5 * (Q3 - Q1) if Q1 - 1.5 * (Q3 - Q1) > 0 else 0 
upper_extreme = Q3 + 1.5 * (Q3 - Q1)

In [62]:
display(f"Q1: {Q1}", 
        f"median: {median}", 
        f"Q3: {Q3}", 
        f"lower_extreme: {lower_extreme}", 
        f"upper_extreme: {upper_extreme}")

'Q1: 0.3492000000019697'

'median: 1.8710147305133815'

'Q3: 8.652146501987207'

'lower_extreme: 0'

'upper_extreme: 21.106566254965063'

In [63]:
df_characteristics_aob_weather["AoB_Category"] = ''

df_characteristics_aob_weather.loc[
    (df_characteristics_aob_weather['Total_AoB'] > lower_extreme) &
    (df_characteristics_aob_weather['Total_AoB'] <= Q1), 'AoB_Category'] = 'Very Low'

df_characteristics_aob_weather.loc[
    (df_characteristics_aob_weather['Total_AoB'] > Q1) &
    (df_characteristics_aob_weather['Total_AoB'] <= median), 'AoB_Category'] = 'Low'

df_characteristics_aob_weather.loc[
    (df_characteristics_aob_weather['Total_AoB'] > median) &
    (df_characteristics_aob_weather['Total_AoB'] <= Q3), 'AoB_Category'] = 'Moderate'

df_characteristics_aob_weather.loc[
    (df_characteristics_aob_weather['Total_AoB'] > Q3) &
    (df_characteristics_aob_weather['Total_AoB'] <= upper_extreme), 'AoB_Category'] = 'High'

df_characteristics_aob_weather.loc[
    (df_characteristics_aob_weather['Total_AoB'] > upper_extreme), 'AoB_Category'] = 'Very High'

df_characteristics_aob_weather['AoB_Category'].value_counts()

Very Low     3520
Moderate     3519
Low          3517
Very High    2065
High         1453
Name: AoB_Category, dtype: int64

In [64]:
df_characteristics_aob_weather.columns = ['total_aob', 'fire_status', 'satelite', 'lat_mean', 'lat_std',
       'lon_mean', 'lon_std', 't21_mean', 't21_std', 't31_mean', 't31_std', 'frp_mean', 'frp_std', 'conf_mean', 'conf_std', 'temp_mean', 'temp_std', 'convective_energy_mean', 'convective_energy_std', 'dewpoint_temp_mean',
       'dewpoint_temp_std', 'total_precipitation_mean', 'total_precipitation_std', 'total_cloud_cover_mean', 
       'total_cloud_cover_std', 'high_veg_cover_mean', 'low_veg_cover_mean', 'soil_water_mean', 'soil_water_std', 
       'wind_speed_mean', 'wind_speed_std', 'category_aob']

## Metadata

In [65]:
display(df_characteristics_aob_weather.head())
display(df_characteristics_aob_weather.shape)

total_aob fire_status satelite  lat_mean  \
UID_Fire REF_ID         Date                                                   
51       BC-2014-G40112 2014-10-24   0.245498     primary        T    54.281   
56       AB-2011-CWF240 2012-05-12   1.594830     primary        T    51.160   
68       BC-2014-C50011 2014-04-12   0.145104     primary        A    52.099   
                        2014-04-12   0.145104     primary        T    52.106   
84       BC-2014-C40028 2014-04-28   0.098025     primary        A    51.193   

                                     lat_std  lon_mean   lon_std  t21_mean  \
UID_Fire REF_ID         Date                                                 
51       BC-2014-G40112 2014-10-24  0.000000 -124.8510  0.000000     28.85   
56       AB-2011-CWF240 2012-05-12  0.005657 -114.8985  0.000707     38.65   
68       BC-2014-C50011 2014-04-12  0.000000 -123.4020  0.000000     29.85   
                        2014-04-12  0.000000 -123.4070  0.000000     38.25   
84       BC-2014-C40028 2014-04-28  0.000000 -121.5740  0.000000     90.65   

                                     t21_std  t31_mean  ...  \
UID_Fire REF_ID         Date                            ...   
51       BC-2014-G40112 2014-10-24  0.000000      2.85  ...   
56       AB-2011-CWF240 2012-05-12  0.707107     23.05  ...   
68       BC-2014-C50011 2014-04-12  0.000000     14.35  ...   
                        2014-04-12  0.000000     14.55  ...   
84       BC-2014-C40028 2014-04-28  0.000000      9.25  ...   

                                    total_precipitation_std  \
UID_Fire REF_ID         Date                                  
51       BC-2014-G40112 2014-10-24                 0.000000   
56       AB-2011-CWF240 2012-05-12                 0.000000   
68       BC-2014-C50011 2014-04-12                 0.000012   
                        2014-04-12                 0.000012   
84       BC-2014-C40028 2014-04-28                 0.000047   

                                    total_cloud_cover_mean  \
UID_Fire REF_ID         Date                                 
51       BC-2014-G40112 2014-10-24                0.838529   
56       AB-2011-CWF240 2012-05-12                0.314304   
68       BC-2014-C50011 2014-04-12                0.377443   
                        2014-04-12                0.377443   
84       BC-2014-C40028 2014-04-28                0.365491   

                                    total_cloud_cover_std  \
UID_Fire REF_ID         Date                                
51       BC-2014-G40112 2014-10-24               0.187564   
56       AB-2011-CWF240 2012-05-12               0.386498   
68       BC-2014-C50011 2014-04-12               0.275385   
                        2014-04-12               0.275385   
84       BC-2014-C40028 2014-04-28               0.313924   

                                    high_veg_cover_mean  low_veg_cover_mean  \
UID_Fire REF_ID         Date                                                  
51       BC-2014-G40112 2014-10-24             0.981841            0.000504   
56       AB-2011-CWF240 2012-05-12             0.934903            0.065097   
68       BC-2014-C50011 2014-04-12             0.969969            0.028169   
                        2014-04-12             0.969969            0.028169   
84       BC-2014-C40028 2014-04-28             0.953703            0.046297   

                                    soil_water_mean  soil_water_std  \
UID_Fire REF_ID         Date                                          
51       BC-2014-G40112 2014-10-24         0.394290        0.005209   
56       AB-2011-CWF240 2012-05-12         0.375507        0.004740   
68       BC-2014-C50011 2014-04-12         0.323027        0.001149   
                        2014-04-12         0.323027        0.001149   
84       BC-2014-C40028 2014-04-28         0.317802        0.001737   

                                    wind_speed_mean  wind_speed_std  \
UID_Fire REF_ID         Date                              

(14074, 32)

In [66]:
path_gold = os.path.abspath(
        os.path.join(os.getcwd(), "../data/processed/gold.csv"))
df_characteristics_aob_weather.to_csv(path_gold, index = True)

## <h3 align="center"> Copyright © 2022 - All rights reserved by the author.<h3/>